In [1]:
import pandas as pd
import tkinter as tk
from tkinter import simpledialog
from CoolProp.CoolProp import PropsSI
import numpy as np

### Function to get the initial values from the user

In [2]:

def ask_for_inputs():
    root = tk.Tk()
    root.withdraw()
    F = float(simpledialog.askstring("Input", "Enter the initial flow (kg):"))
    xf = float(simpledialog.askstring("Input", "Enter the concentration of the feed flow (% w/w):"))
    Tf = float(simpledialog.askstring("Input", "Enter the feed temperature (K):"))
    P1 = float(simpledialog.askstring("Input", "Enter the pressure inside the effect (Pa):"))
    Ps = float(simpledialog.askstring("Input", "Enter the steam pressure (Pa):"))
    xL = float(simpledialog.askstring("Input", "Enter the concentration of the output flow (% w/w):"))

    return xf, Tf, P1, Ps, F, xL


### Material Balance (One evaporator in steady state)

In [3]:
def material_balance(F:float,xf:float,xL:float):
    """
    Calculates the material balance in an evaporator.

    This function determines the liquor flow (L) and vapor flow (V) exiting an evaporator 
    based on the initial flow (F) and the concentrations of the inlet (xf) and outlet (xL) flows.
    Args:
        F (float): Initial flow in kg.
        xf (float): Concentration of the flow in % w/w.
        xL (float): Concentration of the output flow in % w/w.

    Returns:
        L (float): Liquor flow out of the evaporator in kg. 
        V (float): Vapor flow out of the evaporator in kg.
    """    
    L=F*xf/xL
    V=F-L
    print(f"Feed flow (F): {F} kg\nFeed concentration (xf): {xf} % w/w\nLiquor concentration (xL): {xL} % w/w")
    print(f"Liquor flow (L): {L:.2f} kg\nVapor flow (V): {V:.2f} kg")
    return L,V

### Boling_Point_Elevation 
Fundamentals of Salt Water Desalination (Appendix B1)

In [6]:
def Boling_Point_Elevation(xf:float,P1: float):
    """
    Calculates the boiling point elevation (BPE) and the corresponding temperature inside a system (T1) 
    based on a given concentration of a solute (salt) and the absolute pressure of the system
    Args:
        xf (float): Concentration of the flow (% w/w).
        Tf (float): Feed temperature
        P1 (float): Absolute pressure inside the effect (Pa)

    Returns:
        BPE (float): Boiling point elevation (K). 
        T1 (float): Temperature inside the effect (K).
    """
    
    #Compute water properties
            #Q is quality, where 0 is saturated liquid and 1 is saturated vapor
    Tw= PropsSI('T', 'P', P1, 'Q', 0, 'Water')-273.15 #Boiling point of pure water at P1 (°C)
    A=8.325E-2+(1.883E-4*Tw)+(4.02E-6*(Tw**2))
    B=-7.625e-4+(9.02e-5*Tw)-(5.2e-7*(Tw**2))
    C=1.522E-4-(3E-6*Tw)-(3E-8*(Tw**2))
    BPE=A*xf+(B*(xf**2))+(C*(xf**3))
    T1= (Tw+BPE)+273.15 #Boiling point of the solution at P1 (K)
    print(f"Boiling Point Elevation (BPE): {BPE:.2f} °C\nTemperature inside the effect (T1): {T1:.2f} K")
    return BPE, T1


### Energy Balance (One evaporator in steady state)

In [7]:
def enthalpy (T,P,sF,hfe):
    hf=PropsSI('H', 'T', T ,'P', P, 'Water')
    a1, a2, a3, a4, a5,a6,a7,a8,a9,a10 = (-2.348e4, 3.152e5, 2.803e6, -1.446e5, 7.826e03,-4.417e1,2.139e-1,-1.991e4,2.778e4,9.728e1)
    tf=T-273.15
    hf=hf-(sF*(a1+(a2*sF)+(a3*(sF**2))+(a4*(sF**3))+(a5*tf)+(a6*(tf**2))+(a7*(tf**3))+(a8*tf*sF)+(a9*tf*(sF**2))+(a10*sF*(tf**2))))
    err=abs((hf-hfe)/hfe)*100
    print(f"Error compared to IAPWS {err:.2f}%")
    return hf

In [68]:
HF1=enthalpy (353,101325,0.1,0.294363423e6)

Error compared to iapws 0.43%


293092.6155173713

In [20]:
def energy_balance(xf:float,Tf: float,P1: float,T1: float,Ps:float,F:float,V: float,L: float,xL:float):
    """

    Calculates the energy balance in an evaporator.

    This function computes the temperature of the steam vapor (Ts) and the mass flow of steam (S) 
    based on the flow concentrations, temperatures, and pressures involved in the evaporation process.

    Args:
        xf (float): Concentration of the flow (% w/w).
        Tf (float): Feed temperature
        P1 (float): Pressure inside the effect (Pa)
        T1 (float): Temperature inside the effect (Pa)
        Ps (float): Steam pressure (Pa)
        L (float): Liquor flow out of the evaporator (kg).
        xL (float): Concentration of the output flow in % w/w. 
        V (float): Vapor flow out of the evaporator (kg). 
        F (float): Initial flow (kg).
    Returns:
        Ts (float): Temperature os the steam vapor (K).
        S (float): steam mass flow (kg).
    """
    #STEAM PROPERTIES
    Ts= PropsSI('T', 'P', Ps, 'Q', 0, 'Water')  #Boiling point of pure water at Ps (K)
    H_liq = PropsSI('H', 'P', Ps, 'Q', 0, 'Water')
    H_vap = PropsSI('H', 'P', Ps, 'Q', 1, 'Water')
    l_heat=H_vap - H_liq    #Latent heat of water vapor at Ps (J/kg)
    
    #Enthalpies of the process
    Hv=PropsSI('H', 'P', P1, 'Q', 1, 'Water') # Produced vapor at P1,T1 without solute
    hf=PropsSI('H', 'T', Tf, 'P', 101325, 'Water')
    hL=PropsSI('H', 'T', T1, 'P', P1, 'Water')#Liquor entalphy at T1,xL
    a1, a2, a3, a4, a5,a6,a7,a8,a9,a10 = (-2.348e4, 3.152e5, 2.803e6, -1.446e5, 7.826e03,-4.417e1,2.139e-1,-1.991e4,2.778e4,9.728e1)
    sF=xf/100
    sL=xL/100
    tf=Tf-273.15
    t1=T1-273.15
    hf=hf-(sF*(a1+(a2*sF)+(a3*(sF**2))+(a4*(sF**3))+(a5*tf)+(a6*(tf**2))+(a7*(tf**3))+(a8*tf*sF)+(a9*tf*(sF**2))+(a10*sF*(tf**2))))
    hL=hL-(sL*(a1+(a2*sL)+(a3*(sL**2))+(a4*(sL**3))+(a5*t1)+(a6*(t1**2))+(a7*(t1**3))+(a8*t1*sL)+(a9*t1*(sL**2))+(a10*sL*(t1**2))))
    # Steam mass flow calculation
    S = (L * hL + V * Hv - F * hf) / l_heat
    A=3054 #cm^2
    U=((S*l_heat)/(A*(Ts-T1)))  #J/K*cm^2

    print(f"Steam mass flow (S): {S:.2f} kg\nTemperature of the steam (Ts): {Ts:.2f} K")
    print(f"The overall heat transfer coefficient (U): {U:.2f} J/K*cm^2")
    return S, Ts,U


In [14]:
#xf, Tf, P1, Ps, F, xL = ask_for_inputs()
#L,V=material_balance(F,xf,xL)
#BPE, T1=Boling_Point_Elevation(xf,P1)
#S,Ts=energy_balance(xf,Tf,P1,T1,Ps,F,V,L)

### Some experimental data to see the performance of the model

#### Preliminary Experimental Data (atmospheric pressure)

In [21]:

F=20.4
xf=5
Tf=298.15
P1=74000 #Atmospheric Pressure
Ps=131000
xL=11.2
L,V=material_balance(F,xf,xL)
BPE, T1=Boling_Point_Elevation(xf,P1)
S,Ts,U=energy_balance(xf,Tf,P1,T1,Ps,F,V,L,xL)


Feed flow (F): 20.4 kg
Feed concentration (xf): 5 % w/w
Liquor concentration (xL): 11.2 % w/w
Liquor flow (L): 9.11 kg
Vapor flow (V): 11.29 kg
Boiling Point Elevation (BPE): 0.70 °C
Temperature inside the effect (T1): 365.25 K
Steam mass flow (S): 23.17 kg
Temperature of the steam (Ts): 380.48 K
The overall heat transfer coefficient (U): 1114.48 J/K*cm^2


In [30]:
Patm_exp = {
    "P1 (Pa)": P1, # Pressure in the effect (Pa)
    "F (kg)": 20.4,   # Feed flow (kg)
    "xf (% w/w)": 5.0,   # Feed concentration (% w/w)
    "xL (% w/w)": 11.2,   # Liquor concentration (% w/w)
    "L (kg)": 7.242,    # Liquor flow (kg)
    "V (kg)": 13.1,    # Vapor flow (kg)
    "S (kg)": 17.5     # Steam mass flow (kg)
}

In [31]:
Patm_exp = pd.DataFrame([Patm_exp])
data = {
    "P1 (Pa)": P1, # Pressure in the effect (Pa)
    "F (kg)": F,
    "xf (% w/w)": xf,
    "xL (% w/w)": xL,
    "L (kg)": L,
    "V (kg)": V,
    "S (kg)": S
}
Patm_exp.loc[len(Patm_exp)] = data

In [19]:
Patm_exp["(L+V)-F"]=(Patm_exp["L (kg)"]+Patm_exp["V (kg)"])-Patm_exp["F (kg)"]
Patm_exp

,P1 (Pa),F (kg),xf (% w/w),xL (% w/w),L (kg),V (kg),S (kg),(L+V)-F
0,74000,20.4,5.0,11.2,7.242000,13.100000,17.500000,-0.058
1,74000,20.4,5.0,11.2,9.107143,11.292857,23.324553,0.000


Discrepancies Observed in Calculations and Preliminary Experimental Data (atmospheric pressure)

In [11]:
errors = (abs(Patm_exp.iloc[0,4:7] - Patm_exp.iloc[1,4:7])/Patm_exp.iloc[0,4:7])*100
n_columns = ['% Error_L', '% Error_V', '% Error_S'] 
d_errors = pd.DataFrame([errors.values], columns=n_columns)
# Imprimir el mensaje sobre el ensayo
print("\nThis is calculated for the given data:")
print(f"F = {F} kg, xf = {xf}% w/w, Tf = {Tf} K, P1 = {P1} Pa, Ps = {Ps} Pa, xL = {xL}% w/w")
d_errors


This is calculated for the given data:
F = 20.4 kg, xf = 5% w/w, Tf = 298.15 K, P1 = 74000 Pa, Ps = 131000 Pa, xL = 11.2% w/w


,% Error_L,% Error_V,% Error_S
0,25.754527,13.794984,33.28316


#### Preliminary Experimental Data (vaccum pressure)

In [12]:
F2=15.912
xf2=5
Tf2=298.15
P12=46909
Ps2=131000
xL2=13.3
L2,V2=material_balance(F2,xf2,xL2)
BPE2, T12=Boling_Point_Elevation(xf2,P12)
S2,Ts2=energy_balance(xf2,Tf2,P12,T12,Ps2,F2,V2,L2)

Feed flow (F): 15.912 kg
Feed concentration (xf): 5 % w/w
Liquor concentration (xL): 13.3 % w/w
Liquor flow (L): 5.98 kg
Vapor flow (V): 9.93 kg
Boiling Point Elevation (BPE): 0.66 °C
Temperature inside the effect (T1): 353.55 K
Steam mass flow (S): 18.06 kg
Temperature of the steam (Ts): 380.48 K


In [13]:
Vaccumexp = {
    "P1 (Pa)": P12, # Pressure in the effect (Pa)
    "F (kg)": F2,   # Feed flow (kg)
    "xf (% w/w)": xf2,   # Feed concentration (% w/w)
    "xL (% w/w)": 13.3,   # Liquor concentration (% w/w)
    "L (kg)": 7.956,    # Liquor flow (kg)
    "V (kg)": 6.3,    # Vapor flow (kg)
    "S (kg)": 11.4     # Steam mass flow (kg)
}
Vaccum = pd.DataFrame([Vaccumexp])
Vaccum_model = {
    "P1 (Pa)": P12, # Pressure in the effect (Pa)
    "F (kg)": F2,   # Feed flow (kg)
    "xf (% w/w)": xf2,   # Feed concentration (% w/w)
    "xL (% w/w)": xL2,   # Liquor concentration (% w/w)
    "L (kg)": L2,    # Liquor flow (kg)
    "V (kg)": V2,    # Vapor flow (kg)
    "S (kg)": S2     # Steam mass flow (kg)
}
Vaccum.loc[len(Vaccumexp)] = Vaccum_model 


In [14]:
Vaccum["(L+V)-F"] = (Vaccum["L (kg)"] + Vaccum["V (kg)"]) - Vaccum["F (kg)"]
Vaccum

,P1 (Pa),F (kg),xf (% w/w),xL (% w/w),L (kg),V (kg),S (kg),(L+V)-F
0,46909,15.912,5,13.3,7.956000,6.300000,11.400000,-1.656
7,46909,15.912,5,13.3,5.981955,9.930045,18.055327,0.000


Discrepancies Observed in Calculations and Preliminary Experimental Data (vaccum)

In [93]:
errors_v = (abs(Vaccum.iloc[0, 4:7] - Vaccum.iloc[1, 4:7]) / Vaccum.iloc[0, 4:7]) * 100

d_errorsv = pd.DataFrame([errors_v.values], columns=n_columns)
d_errorsv

,% Error_L,% Error_V,% Error_S
0,24.81203,57.619764,58.380058
